https://www.tensorflow.org/tutorials/customization/basics

텐서플로 2.0부터 값이 나오도록 바뀌었다. .numpy하면 값이 튀어나오는데 이러한 것을 eagerly executing이라고 한다.

어떤 연산하는 것을 내부적으로 그려주는 것을 컴퓨테이션 그래프라고 한다. 이걸 해두면, 전체적인 계산을 빠르게 할 수 있다.

텐서플로1에서는 세션이라는 것이 있는데, 거기서 컴퓨테이션 그래프를 그려준다.

1. 함수형 패러다임: (선언형 패러다임) 파이썬 멀티패러다임... 
2. 명령형 패러다임: 쉽게 확인할 수 있는 패러다임 (tf2.0)

# computational graph

연산하는 것을 다 그래프로 만드는 것을 말한다. 이건 네트워크와 비슷하다.

1. 중간과정까지 결과값을 다 알 수 있다.
2. back propagation까지 다 가능하다. 미분값가지 다 계산할 수 있다.

리스코프 치환원칙: 부모에게서 같은게 있으면, 자식에게서 실행할 수 있다.
https://ko.wikipedia.org/wiki/%EB%A6%AC%EC%8A%A4%EC%BD%94%ED%94%84_%EC%B9%98%ED%99%98_%EC%9B%90%EC%B9%99

객체지향의 5원칙중의 하나이다. 리스코프 치환원칙... 공부해야 함.

In [1]:
import tensorflow as tf
import numpy as np

x = tf.add(np.array([1, 2, ]), np.array([2, 3, ]))
x

<tf.Tensor: id=2, shape=(2,), dtype=int32, numpy=array([3, 5])>

결과값이 텐서로 나온다. 텐서플로 함수에 집어넣으면 텐서로 나온다. 이렇게 텐서플로와 넘파이는 서로 호환이 잘된다.

In [2]:
y = np.add(tf.constant([1,2]), tf.constant([2, 3]))
y 

array([3, 5])

넘파이 함수에 넣으면, 넘파이로 나오게 된다. 이건 외워야 한다. 이렇듯 데이터도 내부적으로 넘파이로 되어 있지만, 텐서플로에서는 텐서로 계산을 해주는 것이다.

In [3]:
from keras.datasets import mnist

data, labels = mnist.load_data()

Using TensorFlow backend.


data는 넘파이로 되어 있다. 하지만 넘파이를 집어넣어도 텐서로 계산이 되서, 우리가 원하는 결과값이 나온다.

# GPU 가속

In [4]:
tf.test.is_gpu_available()

True

In [5]:
x = tf.random.uniform([3, 3])
x.device.endswith('GPU:0')

True

In [6]:
with tf.device("CPU:0"): # __enter__, __exit__을 자동으로 해준다.
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("CPU:0")

In [7]:
dir(tf.device('GPU:0')) # __enter__와 __exit__가 있다.

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_ctx',
 '_device_name',
 '_stack']

In [8]:
import inspect

t = tf.device('GPU:0')
print(inspect.getsource(t.__exit__))

  def __exit__(self, *ex_info):
    ctx = self._ctx
    old_device_name, old_device_spec, new_device_spec = self._stack[-1]
    if ctx.device_spec is not new_device_spec:
      raise RuntimeError(
          "Exiting device scope without proper scope nesting")
    del self._stack[-1]
    ctx._set_device(old_device_name, old_device_spec)  # pylint: disable=protected-access



# tf.data (텐서로 데이터를 관리!!!)

이런 식으로 디바이스를 정해서 지맘대로 gpu를 할당할 수 있다. 케라스는 지가 알아서 gpu를 할당시켜 준다.

텐서2.0에서 넘파이가 아니라, 아예 데이터를 텐서로 관리하는 기법이 나왔다. 넘파이를 텐서로 변환하는 과정에서 리소스가 들기 때문에 이게 유용하다. 이것이 바로 tf.data 이다.

In [9]:
dir(tf.data.Dataset) 

['_GeneratorState',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add_variable_with_custom_getter',
 '_apply_options',
 '_as_serialized_graph',
 '_checkpoint_dependencies',
 '_component_metadata',
 '_consumers',
 '_deferred_dependencies',
 '_flat_shapes',
 '_flat_structure',
 '_flat_types',
 '_from_components',
 '_functions',
 '_gather_saveables_for_checkpoint',
 '_graph',
 '_handle_deferred_dependencies',
 '_has_captured_ref',
 '_inputs',
 '_is_graph_tensor',
 '_list_extra_dependencies_for_serialization',
 '_list_functions_for_serialization',
 '_lookup_dependency',
 '_maybe_initialize_trackable',
 '_name_based_att

next로 레이지기법써서 메모리관리한다. filter, apply, batch등이 있다. for문 안쓰고 관리하는 기법이다.

### from_tensor_slices

In [2]:
type(tf.data.Dataset.from_tensor_slices) # from 붙는 애들이니까 클래스 매소드이다. (클래스 매소드가 뭐지?ㅅㅂ)

function

In [3]:
t = tf.data.Dataset.from_tensor_slices([1, 2, 3]) # 쪼개기 좋은 애들
t2 = tf.data.Dataset.from_tensors([1, 2, 3]) # 
print(t)
print(t2)

<TensorSliceDataset shapes: (), types: tf.int32>
<TensorDataset shapes: (3,), types: tf.int32>


In [4]:
t.numpy() # 값이 안 니온다.

AttributeError: 'TensorSliceDataset' object has no attribute 'numpy'

값을 만드는 3가지 방법

1. 리터럴
2. 인스턴스화
3. 팩토리 매소드



In [5]:
# t.shuffle(1)
t.take(2) # 어떤 객체로 되어 있기 때문에 눈에 안보인다.

for i in t.take(2): # n개를 뽑아라.
    print(i)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)


In [6]:
for i in t.batch(2): # 이렇게 텐서로 최적화된 것을 지원한다.
    print(i)

tf.Tensor([1 2], shape=(2,), dtype=int32)
tf.Tensor([3], shape=(1,), dtype=int32)


In [7]:
t = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5])

for i in t.map(lambda x: x+2):
    print(i)

for i in t.map(lambda x: x+2):

If this is a lambda function, the error may be avoided by creating the lambda in a standalone statement.
for i in t.map(lambda x: x+2):

If this is a lambda function, the error may be avoided by creating the lambda in a standalone statement.
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)


In [16]:
dir(t)

['_GeneratorState',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add_variable_with_custom_getter',
 '_apply_options',
 '_as_serialized_graph',
 '_checkpoint_dependencies',
 '_component_metadata',
 '_consumers',
 '_deferred_dependencies',
 '_flat_shapes',
 '_flat_structure',
 '_flat_types',
 '_from_components',
 '_functions',
 '_gather_saveables_for_checkpoint',
 '_graph',
 '_graph_attr',
 '_handle_deferred_dependencies',
 '_has_captured_ref',
 '_inputs',
 '_is_graph_tensor',
 '_list_extra_dependencies_for_serialization',
 '_list_functions_for_serialization',
 '_lookup_dependency',
 '_maybe_initialize_trackable',
 

# 분류, 디텍션, 세그멘테이션_패션 Mnist 실습해보기

1. 분류: classification 전체 이미지중에 특정 이미지가 뭔지 찾는 거다.
2. detection: 이미지 내에서 뭔가가 있는 것에 직사각형을 그려주는것
3. 세그먼테이션: 이미지에서 고양이 윤곽선을 따라서 선을 그려줌

분류 -> 디텍션 -> 세그먼테이션 순서로 가는 것이다.

In [8]:
import tensorflow as tf
from tensorflow import keras

In [9]:
import numpy as np # 데이터가 넘파이로 되어있음
import matplotlib.pyplot as plt # 결과값 그래프 그릴 때

In [10]:
fashion_mnist = keras.datasets.fashion_mnist

(train_img, train_labels), (test_img, test_labels) = fashion_mnist.load_data()

In [11]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

머신러닝의 목적
★ 데이터로부터 스페시픽한 문제해결을 위한 최적의 모델을 찾는 것

In [12]:
train_img = train_img / 255.0
test_img = test_img / 255.0
# 이렇게 노말라이즈 해준 것이다. 0부터 1 사이 값으로.
# 모델만들때 쓴 방식은 예측할 때도 똑같이 써야 한다.

In [13]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(128, input_shape=(28, 28), activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax')) # 10개 분류!

맨 마지막에 쓰는 활성화 함수는 소프트맥스, 바이너리는 시그모이드.

1. 멀티클래스: 소프트맥스
2. 바이너리: 시그모이드
3. 리그레션 값: x

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


컴파일 전략

컴파일 하는 순간 내부적으로 컴퓨테이션 그래프를 그려준다.

목적과 데이터 형태에 따라 이걸 다르게 하면 된다.

1. loss: 클래시피케이션 하는 문제에서는 categorical_crossentropy를 쓴다. (기본) 근데 원핫 인코딩을 안 한 데이터에서는 sparse_caterogical crossentropy를 쓰면 된다.
2. optimizer: gradient descent 문제를 어떻게 풀까... 이걸 정하는 방식이 옵티마이저이다. 
3. metric: 삼총사 기억하면 된다. 

코드 갖고 있는건 의미가 없으니 외워서 줄줄 나오게 해야 한다.

In [24]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
model.fit(train_img, train_labels) # validation set을 fit에서 지정할 수 있다니...
# 하지만 fit을 사용 안하는게 더 많다. 고급자는...

Train on 60000 samples
60000/60000 [==============================] - 4s 70us/sample - loss: 0.5040 - accuracy: 0.8219s - loss:


In [26]:
a = model.train_on_batch(train_img, train_labels) # loss accuracy
# 이런 방식으로 배치마다 트레인을 다르게 할 수 있다.
# 그리고 그래디언트 디센트를 내 맘대로 할 수 있는 방법도 있다.

In [27]:
dir(a)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

__iter__있으니, for문 안에 넣을 수 있다.

In [33]:
model.evaluate(test_img, test_labels, verbose = 2, workers =4)

10000/1 - 1s - loss: 0.3224 - accuracy: 0.8490


[0.4247963364362717, 0.849]

In [37]:
a = model.fit(train_img, train_labels)
dir(a) # 히스토리가 있다.

Train on 60000 samples
60000/60000 [==============================] - 4s 65us/sample - loss: 0.3402 - accuracy: 0.8758


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_chief_worker_only',
 '_keras_api_names',
 '_keras_api_names_v1',
 'epoch',
 'history',
 'model',
 'on_batch_begin',
 'on_batch_end',
 'on_epoch_begin',
 'on_epoch_end',
 'on_predict_batch_begin',
 'on_predict_batch_end',
 'on_predict_begin',
 'on_predict_end',
 'on_test_batch_begin',
 'on_test_batch_end',
 'on_test_begin',
 'on_test_end',
 'on_train_batch_begin',
 'on_train_batch_end',
 'on_train_begin',
 'on_train_end',
 'params',
 'set_model',
 'set_params',
 'validation_data']

In [38]:
a.history # 딕셔너리 형태로 되어 있는데, 이 결과를 그래프로 그리면 된다.

{'loss': [0.3401643986284733], 'accuracy': [0.87581664]}

In [39]:
b = model.evaluate(test_img, test_labels)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [41]:
len(b) # evaluate는 2개로 할당하는 것이다.

2

In [42]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy')
#               metrics=['accuracy']) # 여기에 따라서, evaluate에서 나오는 결과 개수가 달라진다.

In [44]:
model.fit(train_img, train_labels)
b = model.evaluate(test_img, test_labels)

Train on 60000 samples
60000/60000 [==============================] - 4s 64us/sample - loss: 0.3165
10000/1 [===========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [47]:
b

0.3492695860147476

### 128을 64로 바꾸기

64로 바꾸면 성능이 조금 낮아진다.

fit한 뒤 할당하는 것 잘하기.

In [59]:
model2 = keras.Sequential()
model2.add(keras.layers.Flatten(input_shape=(28, 28)))
model2.add(keras.layers.Dense(64, activation='relu'))
model2.add(keras.layers.Dense(10, activation='softmax')) # 10개 분류!

In [60]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model2.fit(train_img, train_labels, epoch=5)

In [65]:
model2.evaluate(test_img, test_labels, verbose = 2)

10000/1 - 0s - loss: 0.3102 - accuracy: 0.8590


[0.39253191604614257, 0.859]